In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [3]:
## Load dataset
data_path = "../../../rings_varying_radius_c2k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()
NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_GLOBAL = 64
NUM_HIDDEN_LOCAL = 64
NUM_STATS = 64

MCMC_SIZE = 20
BATCH_SIZE = 20
NUM_EPOCHS = 1000
LEARNING_RATE = 1e-3

CUDA = torch.cuda.is_available()
PATH = 'ep-rad-init-z'
DEVICE = torch.device('cuda:1')

noise_sigma = torch.ones(1) * 0.05
if CUDA:
    noise_sigma = noise_sigma.cuda().to(DEVICE)
Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (noise_sigma, N, K, D, MCMC_SIZE)

In [4]:
from local_enc import *
from global_enc import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, NUM_HIDDEN_GLOBAL, NUM_STATS, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)

optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (enc_eta, enc_eta, enc_z)

In [5]:
# enc_z.load_state_dict(torch.load("../weights/enc-z-%s" % PATH))
# enc_eta.load_state_dict(torch.load("../weights/enc-eta-%s" % PATH))

In [6]:
from ag_ep_rad import *
train(models, EUBO_init_z, optimizer, Data, Model_Params, Train_Params)

epoch: 0\1000 (41s),  symKL_DB_eta: 297980.341,  symKL_DB_z: 179744.341,  gap: 477502.696,  loss: -82290.452,  ess: 2.292
epoch: 1\1000 (40s),  symKL_DB_eta: 105570.974,  symKL_DB_z: 156124.142,  gap: 261653.849,  loss: -45820.540,  ess: 2.555
epoch: 2\1000 (41s),  symKL_DB_eta: 45420.031,  symKL_DB_z: 106899.867,  gap: 152288.124,  loss: -37373.618,  ess: 2.991
epoch: 3\1000 (40s),  symKL_DB_eta: 31197.707,  symKL_DB_z: 69592.421,  gap: 100779.665,  loss: -34684.763,  ess: 3.424
epoch: 4\1000 (40s),  symKL_DB_eta: 27412.903,  symKL_DB_z: 44387.657,  gap: 71795.730,  loss: -33483.875,  ess: 3.808
epoch: 5\1000 (40s),  symKL_DB_eta: 24558.602,  symKL_DB_z: 29013.099,  gap: 53567.384,  loss: -33139.283,  ess: 4.126
epoch: 6\1000 (41s),  symKL_DB_eta: 22716.121,  symKL_DB_z: 19772.768,  gap: 42488.896,  loss: -32825.006,  ess: 4.370
epoch: 7\1000 (40s),  symKL_DB_eta: 20523.684,  symKL_DB_z: 13945.091,  gap: 34467.045,  loss: -32491.667,  ess: 4.549
epoch: 8\1000 (40s),  symKL_DB_eta: 198

KeyboardInterrupt: 

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)

In [ ]:
from ag_ep_rad import *

BATCH_SIZE_TEST = 25
Train_Params_Test = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE_TEST, CUDA, DEVICE, PATH)
Model_Params_Test = (noise_sigma, N, K, D, 10)
obs, metric_step, reused = test(models, EUBO_init_z, Data, Model_Params_Test, Train_Params_Test)
(q_mu, _, q_z, _) = reused

In [ ]:
%time plot_samples(obs, q_mu, q_z, K, PATH)

In [ ]:
incremental_gap = symkls_test.cpu().data.numpy()[1:]
M = incremental_gap.shape[0]
overall_gap = np.zeros(M)
for m in range(M):
    overall_gap[m] = incremental_gap[:m+1].sum()

In [ ]:
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(111)
plt.yscale("log")
ax.plot(incremental_gap, label="incremental gap")
ax.plot(overall_gap, label='overall gap')
ax.legend(fontsize=14)
ax.set_xlabel('Steps')